In [2]:
import pandas as pd    
  
# 读取Excel文件    
file_path = '专题统计_实际控制人持股变动.xlsx'  # 请替换为你的Excel文件的路径    
df = pd.read_excel(file_path, engine='openpyxl', dtype={'证券代码': str})    
  
# 创建一个包含要排除的所有关键字的列表  
exclude_keywords = ['国有', '国务院', '人民政府', '中共']  
  
# 使用str.contains()函数检查“实际控制人名称”列中的值是否包含关键字列表中的任何一个关键字，并使用'~'操作符取反，从而筛选出不包含这些关键字的行  
filtered_df = df[~df['实际控制人名称'].str.contains('|'.join(exclude_keywords), na=False)]    
  
# 提取证券代码列并转换为列表    
stock_codes = filtered_df['证券代码'].tolist()    
  
# print(stock_codes)
# 使用列表推导式筛选出以"00"开头的股票代码  
stock_codes_00_prefix = [code for code in stock_codes if code.startswith("00")]  
print(stock_codes_00_prefix)

['000715', '000882', '000620', '002420', '002056', '000795', '000657', '000876', '000926', '000008', '000739', '000990', '000795', '002056', '000786', '000020', '000035', '000042', '000046', '000055', '000056', '000153', '000413', '000416', '000426', '000531', '000534', '000572', '000603', '000617', '000627', '000639', '000665', '000669', '000690', '000712', '000735', '000812', '000838', '000895', '000963', '000967', '000997', '001696', '002002', '000156', '002008', '002026', '000513', '000516', '000517', '002031', '000525', '002043', '002073', '002075', '002078', '002099', '002103', '000626', '000628', '002107', '000637', '002132', '000670', '002146', '000672', '000683', '002157', '002171', '000703', '002174', '002210', '000821', '000863', '000929', '000931', '002019', '002004', '002009', '002016', '002020', '002032', '000166', '002038', '002041', '002048', '002054', '002058', '002062', '002064', '002081', '002067', '002069', '002079', '000576', '002087', '000510', '000040', '002094',

In [3]:
stock_codes = []
# 股票代码列表    
for i in stock_codes_00_prefix:
    stock_codes.append('sz.'+i)
stock_codes

['sz.000715',
 'sz.000882',
 'sz.000620',
 'sz.002420',
 'sz.002056',
 'sz.000795',
 'sz.000657',
 'sz.000876',
 'sz.000926',
 'sz.000008',
 'sz.000739',
 'sz.000990',
 'sz.000795',
 'sz.002056',
 'sz.000786',
 'sz.000020',
 'sz.000035',
 'sz.000042',
 'sz.000046',
 'sz.000055',
 'sz.000056',
 'sz.000153',
 'sz.000413',
 'sz.000416',
 'sz.000426',
 'sz.000531',
 'sz.000534',
 'sz.000572',
 'sz.000603',
 'sz.000617',
 'sz.000627',
 'sz.000639',
 'sz.000665',
 'sz.000669',
 'sz.000690',
 'sz.000712',
 'sz.000735',
 'sz.000812',
 'sz.000838',
 'sz.000895',
 'sz.000963',
 'sz.000967',
 'sz.000997',
 'sz.001696',
 'sz.002002',
 'sz.000156',
 'sz.002008',
 'sz.002026',
 'sz.000513',
 'sz.000516',
 'sz.000517',
 'sz.002031',
 'sz.000525',
 'sz.002043',
 'sz.002073',
 'sz.002075',
 'sz.002078',
 'sz.002099',
 'sz.002103',
 'sz.000626',
 'sz.000628',
 'sz.002107',
 'sz.000637',
 'sz.002132',
 'sz.000670',
 'sz.002146',
 'sz.000672',
 'sz.000683',
 'sz.002157',
 'sz.002171',
 'sz.000703',
 'sz.0

In [7]:
import baostock as bs    
import pandas as pd    
import os  
  
# 登陆系统    
lg = bs.login()    
# 显示登陆返回信息    
print('login respond error_code:' + lg.error_code)    
print('login respond  error_msg:' + lg.error_msg)    
    

  
# 指定存放Excel文件的文件夹路径  
folder_path = "非国企深市的季频盈利能力"  # 替换为你的文件夹路径  
  
# 如果文件夹不存在，创建它  
if not os.path.exists(folder_path):  
    os.makedirs(folder_path)  
  
# 遍历每个股票    
for code in stock_codes:    
    print(f"Processing {code}")  
    all_data_list = []    
    # 遍历每一年的四个季度    
    for year in range(2007, 2024):  # 2024用于包括2023年的数据    
        for quarter in range(1, 5):  # 一年有四个季度    
            print(f"Processing {year} quarter {quarter}")    
            # 季频偿债能力  
            profit_list = []    
            rs_profit = bs.query_profit_data(code=code, year=year, quarter=quarter)    
            
            # 检查是否有数据，如果没有则跳过这一年份和季度  
            if not rs_profit.data:  
                print(f"No data found for {year} quarter {quarter}. Skipping...")  
                continue 
            
            while (rs_profit.error_code == '0') & rs_profit.next():    
                profit_list.append(rs_profit.get_row_data())    
            # 将数据添加到列表中    
            result_cash_flow = pd.DataFrame(profit_list, columns=rs_profit.fields)    
            result_cash_flow['code'] = code    
            result_cash_flow['year'] = year    
            result_cash_flow['quarter'] = quarter    
            all_data_list.append(result_cash_flow)    
              
    # 合并该股票的所有数据    
    all_data = pd.concat(all_data_list, ignore_index=True)    
    # 将该股票的数据输出到单独的Excel文件中  
    excel_file_path = os.path.join(folder_path, f"{code}.xlsx") # 创建该股票的Excel文件路径  
    all_data.to_excel(excel_file_path, index=False) # 将数据保存到该Excel文件中  
    print(f"Data for {code} saved to {excel_file_path}\n") # 输出保存信息  
        
# 登出系统    
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
Processing sz.000715
Processing 2007 quarter 1
Processing 2007 quarter 2
Processing 2007 quarter 3
Processing 2007 quarter 4
Processing 2008 quarter 1
Processing 2008 quarter 2
Processing 2008 quarter 3
Processing 2008 quarter 4
Processing 2009 quarter 1
Processing 2009 quarter 2
Processing 2009 quarter 3
Processing 2009 quarter 4
Processing 2010 quarter 1
Processing 2010 quarter 2
Processing 2010 quarter 3
Processing 2010 quarter 4
Processing 2011 quarter 1
Processing 2011 quarter 2
Processing 2011 quarter 3
Processing 2011 quarter 4
Processing 2012 quarter 1
Processing 2012 quarter 2
Processing 2012 quarter 3
Processing 2012 quarter 4
Processing 2013 quarter 1
Processing 2013 quarter 2
Processing 2013 quarter 3
Processing 2013 quarter 4
Processing 2014 quarter 1
Processing 2014 quarter 2
Processing 2014 quarter 3
Processing 2014 quarter 4
Processing 2015 quarter 1
Processing 2015 quarter 2
Processing 2015 quar

ValueError: No objects to concatenate